In [8]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree


from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.model_selection import KFold,LeaveOneOut, cross_val_score
pd.options.display.float_format = "{:,.2f}".format
import sys
sys.path.append("../")
from src import soporte_predic as sp

In [9]:
df = pd.read_csv('../datos/datos_cluster_kmeans.csv', index_col=0)
df.head(2)

df_cluster3 = df[df["clusters_kmeans"] == 2]
df_cluster3.head()

,Ship Mode,Segment,City,State,Country,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Shipping Cost,Order Priority,clusters_kmeans,Profit_respuesta
9,0.20,0.52,0.00,0.00,0.19,0.19,0.13,0.00,0.19,0.02,0.00,19.12,3.33,2.00,39.28,0.08,2,"-1,862.31"
32,0.05,0.52,0.00,0.02,0.19,0.19,0.22,0.00,0.19,0.07,0.00,10.75,0.67,1.50,31.26,0.30,2,-350.49
37,0.05,0.30,0.00,0.00,0.19,0.19,0.13,0.00,0.20,0.03,0.00,35.93,0.33,2.50,30.54,0.30,2,"-3,839.99"
73,0.15,0.30,0.00,0.00,0.03,0.21,0.06,0.00,0.19,0.02,0.00,15.17,1.33,2.35,26.50,0.30,2,-452.81
83,0.20,0.52,0.01,0.01,0.19,0.19,0.06,0.00,0.19,0.07,0.00,19.66,2.00,1.50,25.59,0.57,2,-630.88


In [10]:
df_cluster3 = df_cluster3.drop(columns=['Shipping Cost','Product ID','Product Name','clusters_kmeans','Sub-Category','Region','Segment','State'])

In [11]:
# instanciamos la clase de regresión lineal
analisis = sp.AnalisisModelosRegresion(dataframe=df_cluster3, variable_dependiente='Profit_respuesta')

params_arbol =  {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [10, 50, 100, 200],
    'min_samples_leaf': [10, 50, 100, 200],
    'max_leaf_nodes': [4, 6, 8, 10, 20, 30]
}


In [12]:

# llamamos a la función para ajustar el modelo
analisis.ajustar_modelo(modelo='tree', param_grid=params_arbol, scoring="neg_mean_squared_error")

# llamamos al metodo para obtener los resultaods de las predicciones
df_resultados = analisis.obtener_resultados()
df_resultados

,Real,Predicho,Conjunto,Modelo,Residuos
12759,-579.90,-562.78,Train,tree,-17.12
5701,-387.26,-649.84,Train,tree,262.58
2216,-312.38,-418.26,Train,tree,105.89
18669,-384.72,-438.59,Train,tree,53.87
2397,-996.78,-649.84,Train,tree,-346.94
...,...,...,...,...,...
109,"-2,211.16","-1,184.46",Test,tree,"-1,026.71"
1235,-694.29,-910.33,Test,tree,216.03
1382,"-1,850.95",-649.84,Test,tree,"-1,201.11"
1919,-297.68,-591.83,Test,tree,294.14


In [13]:
analisis.calcular_metricas("tree")

,train,test
R2,0.50,0.36
MAE,191.27,253.71
MSE,"145,319.35","168,416.87"
RMSE,381.21,410.39


In [14]:
# instanciamos la clase de regresión lineal
analisis_random_forest = sp.AnalisisModelosRegresion(dataframe=df_cluster3, variable_dependiente='Profit_respuesta')
params_rf = {
    'n_estimators': [35,50,65,80,100,120,150,200],    #número de árboles que queremos que haga
    'max_features': [4,8,10,13,25,35,45,50],
    'max_depth': [6,7,8],
    'min_samples_split': [25,30,35],
    'min_samples_leaf': [6,7,8], 
    'max_leaf_nodes': [8,9,10]
}

# llamamos a la función para ajustar el modelo
analisis_random_forest.ajustar_modelo(modelo='random_forest', scoring="neg_mean_squared_error", param_grid=params_rf)

# llamamos al metodo para obtener los resultaods de las predicciones
df_resultados_analisis_random_forest = analisis_random_forest.obtener_resultados()
df_resultados_analisis_random_forest.head(3)

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


,Real,Predicho,Conjunto,Modelo,Residuos
12759,-579.90,-552.25,Train,random_forest,-27.65
5701,-387.26,-585.82,Train,random_forest,198.56
2216,-312.38,-432.08,Train,random_forest,119.70


In [15]:
analisis_random_forest.calcular_metricas("random_forest")

,train,test
R2,0.56,0.49
MAE,185.12,239.23
MSE,"126,788.85","134,056.24"
RMSE,356.07,366.14


In [ ]:
# instanciamos la clase de regresión lineal
analisis_gradient_boosting = sp.AnalisisModelosRegresion(dataframe=df_cluster3, variable_dependiente='Profit_respuesta')
params_gb = {
    'n_estimators': [100, 200],  
    'learning_rate': [0.01, 0.05],  
    'max_depth': [3, 4, 5],  
    'min_samples_split': [5, 10, 20],  
    'min_samples_leaf': [2, 4, 8],  
    'subsample': [0.8, 0.9]}

# llamamos a la función para ajustar el modelo
analisis_gradient_boosting.ajustar_modelo(modelo='gradient_boosting',param_grid = params_gb, scoring="neg_mean_squared_error")

# llamamos al metodo para obtener los resultaods de las predicciones
df_resultados_gradient_boosting = analisis_gradient_boosting.obtener_resultados()
df_resultados_gradient_boosting.head(3)

,Real,Predicho,Conjunto,Modelo,Residuos
12759,-579.90,-521.13,Train,gradient_boosting,-58.77
5701,-387.26,-576.37,Train,gradient_boosting,189.11
2216,-312.38,-400.95,Train,gradient_boosting,88.58


In [17]:
analisis_gradient_boosting.calcular_metricas("gradient_boosting")

,train,test
R2,0.86,0.68
MAE,138.63,192.76
MSE,"40,887.80","85,580.33"
RMSE,202.21,292.54


In [ ]:
# instanciamos la clase de regresión lineal
analisis_xgboost= sp.AnalisisModelosRegresion(dataframe=df_cluster3, variable_dependiente='Profit_respuesta')

params_xgb = {
    'n_estimators': [100, 200],  
    'learning_rate': [0.01, 0.05],  
    'max_depth': [3, 4],  
    'min_child_weight': [3, 5], 
    'subsample': [0.7, 0.8],  # Reducir el porcentaje de muestras usadas.
    'colsample_bytree': [0.7, 0.8],  # Reducir características para limitar complejidad.
    'reg_alpha': [0, 0.01],  # Regularización L1 para controlar el tamaño de los coeficientes.
    'reg_lambda': [1, 10],  # Regularización L2 para mayor robustez.
}


# llamamos a la función para ajustar el modelo
analisis_xgboost.ajustar_modelo(modelo='xgboost',param_grid=params_xgb, scoring="neg_mean_squared_error")

# llamamos al metodo para obtener los resultaods de las predicciones
df_resultados_xgboost = analisis_xgboost.obtener_resultados()
df_resultados_xgboost.head(3)

,Real,Predicho,Conjunto,Modelo,Residuos
12759,-579.90,-523.63,Train,xgboost,-56.27
5701,-387.26,-520.16,Train,xgboost,132.90
2216,-312.38,-336.50,Train,xgboost,24.13


In [19]:
analisis_xgboost.calcular_metricas("xgboost")

,train,test
R2,0.90,0.65
MAE,118.76,193.47
MSE,"27,874.13","91,270.63"
RMSE,166.96,302.11
